# D599 Task 3


In [60]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

df = pd.read_csv("Megastore Dataset.csv")
df.head()
df.describe(include="all")

,OrderID,ProductName,Quantity,InvoiceDate,UnitPrice,TotalCost,Country,DiscountApplied,OrderPriority,Region,Segment,ExpeditedShipping,PaymentMethod,CustomerOrderSatisfaction
count,8234.000000,8234,8234.000000,8234,8234,8234,8234,8234,8234,8234,8234,8234,8234,8234
unique,NaN,1562,NaN,441,86,443,1,2,2,2,2,2,2,5
top,NaN,RABBIT NIGHT LIGHT,NaN,10/11/2011 14:52,$1.65,$15.00,United States,No,High,Northeast,Consumer,Yes,PayPal,Prefer not to answer
freq,NaN,75,NaN,259,1033,596,8234,8215,4320,4768,4166,4295,4480,1880
mean,560874.506923,NaN,13.705125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,13082.500625,NaN,21.494536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,536370.000000,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,549274.000000,NaN,6.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,563502.000000,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,571864.000000,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Part 1: Research Question

My proposed research question is "Which products are most frequently purchased together at the Allias superstore?"

The goal of this analysis is to uncover rules of association between our products and use that data to create actionable insights regarding product bundling and cross-product marketing.

## Part 2: Market Basket Justification

The Apriori algorithm searches a dataset for frequently occuring combinations between products. It searches through a data base and forms rules such as Printer ink -> printer paper. These rules include calculations of lift, support, and confidence, and illustrate the liklihood that these products are frequently purchased together. The apriori algorithm starts with finding the support, or liklihood of customers purchasing the item, for each individual item and then starts pairing off groups of products and pruning the pairs without enough support. The expected outcome is a ranked list of rules showing all the products with overwhelming support that they are frequently purchased together.

Below is an example of one transaction in the data set.

In [61]:
df[df["OrderID"] == 536370]["ProductName"]

0             INFLATABLE POLITICAL GLOBE 
1          SET2 RED RETROSPOT TEA TOWELS 
2         PANDA AND BUNNIES STICKER SHEET
3           RED TOADSTOOL LED NIGHT LIGHT
4      VINTAGE HEADS AND TAILS CARD GAME 
5                        STARS GIFT TAPE 
6          VINTAGE SEASIDE JIGSAW PUZZLES
7     ROUND SNACK BOXES SET OF4 WOODLAND 
8                 MINI PAINT SET VINTAGE 
9              MINI JIGSAW CIRCUS PARADE 
10                   MINI JIGSAW SPACEBOY
11                    SPACEBOY LUNCH BOX 
12               CIRCUS PARADE LUNCH BOX 
13                LUNCH BOX I LOVE LONDON
14        CHARLOTTE BAG DOLLY GIRL DESIGN
15             ALARM CLOCK BAKELIKE GREEN
16              ALARM CLOCK BAKELIKE RED 
17              ALARM CLOCK BAKELIKE PINK
18        SET 2 TEA TOWELS I LOVE LONDON 
Name: ProductName, dtype: object

One assumption that market basket analysis makes is individual independence. It assumes that every transaction is independent of one another and not influenced by sales, seasonal habits, etc. It relies on the fact that the data is exactly how it appears in the data set with no influencing outside factors.

## Part 3: Data Prep and Analysis

#### C1: Below I will encode and transform 4 variables. My two ordinal variables are OrderPriority and CustomerSatisfaction and my two nominal variables are Segment, and PaymentMethod.

In [62]:
df_encoded = pd.DataFrame(data = df)

#Ordinal encoding of ordinal variables
priority_map = {"Low": 1, "Medium": 2, "High": 3, "Critical": 4}
df_encoded["OrderPriority_Encoded"] = df_encoded["OrderPriority"].map(priority_map)

satisfaction_map = {'Very Dissatisfied': 1, 'Dissatisfied': 2, 'Prefer not to answer': 3, 'Satisfied': 4, 'Very Satisfied': 5}
df_encoded["CustomerSatisfaction_Encoded"] = df_encoded["CustomerOrderSatisfaction"].map(satisfaction_map)

#One hot encoding of nominal variables
df_encoded = pd.get_dummies(df_encoded, columns=["Segment", "PaymentMethod"], drop_first=False)

#Exporting encoded dataset to csv
df_encoded.to_csv("non_transactional_encoded.csv", index=False)

df_encoded.head()


,OrderID,ProductName,Quantity,InvoiceDate,UnitPrice,TotalCost,Country,DiscountApplied,OrderPriority,Region,ExpeditedShipping,CustomerOrderSatisfaction,OrderPriority_Encoded,CustomerSatisfaction_Encoded,Segment_Consumer,Segment_Corporate,PaymentMethod_Credit Card,PaymentMethod_PayPal
0,536370,INFLATABLE POLITICAL GLOBE,48,12/1/2010 8:45,$0.85,$40.80,United States,Yes,High,Northeast,Yes,Satisfied,3,4,False,True,True,False
1,536370,SET2 RED RETROSPOT TEA TOWELS,18,12/1/2010 8:45,$2.95,$53.10,United States,Yes,High,Northeast,Yes,Satisfied,3,4,False,True,True,False
2,536370,PANDA AND BUNNIES STICKER SHEET,12,12/1/2010 8:45,$0.85,$10.20,United States,Yes,High,Northeast,Yes,Satisfied,3,4,False,True,True,False
3,536370,RED TOADSTOOL LED NIGHT LIGHT,24,12/1/2010 8:45,$1.65,$39.60,United States,Yes,High,Northeast,Yes,Satisfied,3,4,False,True,True,False
4,536370,VINTAGE HEADS AND TAILS CARD GAME,24,12/1/2010 8:45,$1.25,$30.00,United States,Yes,High,Northeast,Yes,Satisfied,3,4,False,True,True,False


For the code above I chose to use ordinal encoding to transform my two ordinal variables. This is supported by the nature of the column, which has a specific order that responses need to be recognized. Due to the ordered nature of these columns, this method was appropriate.

For the nominal variables I used one-hot encoding. This is the convention for nominal variables as they have no specified order. One-hot encoding creates a new boolean column for every values and marks True when the row utilizes that value. This is appropriate for these variables as there is no other way to organize their range of values.

#### C2: Perform Market Basket Analysis

First I will transactionalize the dataset.


In [63]:
#Group products by Order ID
transactions = df_encoded.groupby("OrderID")["ProductName"].apply(list)

#Convert to a new DataFrame
tx_df = pd.DataFrame({"OrderID": transactions.index, "Items": transactions.values})

#Exporting transactionalized data set to csv
tx_df.to_csv("transactional_dataset.csv", index=False)

print(tx_df.head())

   OrderID                                              Items
0   536370  [INFLATABLE POLITICAL GLOBE , SET2 RED RETROSP...
1   536852  [POLKADOT RAIN HAT , VINTAGE HEADS AND TAILS C...
2   536974  [EDWARDIAN PARASOL RED, LUNCH BAG RED RETROSPO...
3   537065  [PARTY TIME PENCIL ERASERS, RED RETROSPOT PURS...
4   537463  [PINK POLKADOT CHILDRENS UMBRELLA, RED RETROSP...


Next I will One-Hot encode the transactionalized data set to prepare it for the Apriori.

In [64]:
#Convert transactions to list of lists
transaction_list = transactions.tolist()

#Transform into one-hot encoded matrix
te = TransactionEncoder()
te_matrix = te.fit(transaction_list).transform(transaction_list)

basket = pd.DataFrame(te_matrix, columns=te.columns_)
basket.head()

,50S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR TLIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC TLIGHT HOLDER STAR LARGE,ZINC TLIGHT HOLDER STARS SMALL
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Next I will run the Apriori algorithm.

In [84]:
#Find the most frequent items
freq_itemsets = apriori(basket, min_support=0.02, use_colnames=True)

#Generate association rules
rules = association_rules(freq_itemsets, metric="lift", min_threshold=1.0)

#Sort by strongest association
rules = rules.sort_values(by="lift", ascending=False)

#Save rules to csv files
rules.to_csv("mba_rules.csv", index=False)
rules.head(3).to_csv("mba_top3.csv", index=False)

rules.head(3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
2854,"(DOLLY GIRL CHILDRENS CUP, SPACEBOY CHILDRENS ...","(SPACEBOY CHILDRENS CUP, DOLLY GIRL CHILDRENS ...",0.029478,0.024943,0.024943,0.846154,33.923077,1.0,0.024208,6.337868,1.000000,0.846154,0.842218,0.923077
2851,"(SPACEBOY CHILDRENS CUP, DOLLY GIRL CHILDRENS ...","(DOLLY GIRL CHILDRENS CUP, SPACEBOY CHILDRENS ...",0.024943,0.029478,0.024943,1.000000,33.923077,1.0,0.024208,inf,0.995349,0.846154,1.000000,0.923077
907,(RECYCLED ACAPULCO MAT TURQUOISE),(RECYCLED ACAPULCO MAT LAVENDER),0.027211,0.022676,0.020408,0.750000,33.075000,1.0,0.019791,3.909297,0.996892,0.692308,0.744200,0.825000


## Part 4: Data Summary and Implication

#### D1: 
The association rules were sorted by lift in descending order. This was chosen due to lifts nature representing the randomness of the association. This can determine if our data is valid or not as a strong lift highly suggests correlation between those rules. By sorting using lift, I am ensuring that the top 3 rules have the strongest correlation between them.

#### D2: 

Support: Percentage of total orders that include this product, or products.

Confidence: Percentage that represents the liklihood that if one specific product is purchased, how likely is it that another specific product is purchased.

Lift: The value representing the randomness of the association. If the lift is below 1, then the products are less likely to be purchased together than normal. If the lift is one, the product has data that is expected and could be random. If the lift is greater than one, that supports a conclusion that the association is positive, and more than random chance.

#### D3:

Our results show strong associations between our top 3 rules. We determined with a very strong lift(30+) that various boys and girls cups are purchased together, as well as turquoise and lavendar mats. These insights can be used to design product bundling or marketing strategies involving those pairings.

#### D4:

Based on the insights gained from this analysis, Allias Megastore should consider bundling the items frequently purchased together, or offering discounts. For example, offering a discount when buying childrens cups, or recommending the turquoise mat when customers purchase the lavendar mat could result in a rise in sales as these items are already purchased together at a very high rate. These actions are supported by the high lift value found through our analysis, which supports that the correlation between these purchases are far from random.